# Pull Data

### Import Libraries

In [ ]:
import os
from dotenv import load_dotenv

import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests

load_dotenv()

### Setup Spotify Credentials

In [15]:
spotify_key = os.getenv('SPOTIFY_KEY')
spotify_secret = os.getenv('SPOTIFY_SECRET')

In [16]:
client_credentials_manager = SpotifyClientCredentials(client_id=spotify_key, 
                                                      client_secret=spotify_secret)

In [17]:
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [18]:
tswift_uri = '06HL4z0CvFAxyc27GXpf02' # Pulled from Taylor Swift Spotify page

### Get List of Albums

In [19]:
tswift_albums = sp.artist_albums(tswift_uri)

In [20]:
name = []
uri = []
for i in tswift_albums['items']:
    name.append(i['name'])
    uri.append(i['uri'])

In [94]:
name[0:5]

["1989 (Taylor's Version) [Deluxe]",
 "1989 (Taylor's Version)",
 "Speak Now (Taylor's Version)",
 'Midnights (The Til Dawn Edition)',
 'Midnights (3am Edition)']

### Get Song Names

In [22]:
song_name = []
song_uri = []
album = []
count = 0
for j in uri:
    
    tracks = sp.album_tracks(j)   
    for i in tracks['items']:
        album.append(name[count])
        song_name.append(i['name'])
        song_uri.append(i['uri'])
    count+=1

["Welcome To New York (Taylor's Version)",
 "Blank Space (Taylor's Version)",
 "Style (Taylor's Version)",
 "Out Of The Woods (Taylor's Version)",
 "All You Had To Do Was Stay (Taylor's Version)",
 "Shake It Off (Taylor's Version)",
 "I Wish You Would (Taylor's Version)",
 "Bad Blood (Taylor's Version)",
 "Wildest Dreams (Taylor's Version)",
 "How You Get The Girl (Taylor's Version)",
 'This Love (Taylor’s Version)',
 "I Know Places (Taylor's Version)",
 "Clean (Taylor's Version)",
 "Wonderland (Taylor's Version)",
 "You Are In Love (Taylor's Version)",
 "New Romantics (Taylor's Version)",
 '"Slut!" (Taylor\'s Version) (From The Vault)',
 "Say Don't Go (Taylor's Version) (From The Vault)",
 "Now That We Don't Talk (Taylor's Version) (From The Vault)",
 "Suburban Legends (Taylor's Version) (From The Vault)",
 "Is It Over Now? (Taylor's Version) (From The Vault)",
 "Bad Blood (feat. Kendrick Lamar) (Taylor's Version)",
 "Welcome To New York (Taylor's Version)",
 "Blank Space (Taylor's Ve

In [95]:
song_name[0:5]

["Welcome To New York (Taylor's Version)",
 "Blank Space (Taylor's Version)",
 "Style (Taylor's Version)",
 "Out Of The Woods (Taylor's Version)",
 "All You Had To Do Was Stay (Taylor's Version)"]

### Get Audio Features

In [23]:
acoustic = []
dance = []
energy = []
instrumental = []
liveness = []
loudness = []
speech = []
tempo = []
valence = []
popularity = []

for i in song_uri:
    feat = sp.audio_features(i)[0]
    acoustic.append(feat['acousticness'])
    dance.append(feat['danceability'])
    energy.append(feat['energy'])
    speech.append(feat['speechiness'])
    instrumental.append(feat['instrumentalness'])
    loudness.append(feat['loudness'])
    tempo.append(feat['tempo'])
    liveness.append(feat['liveness'])
    valence.append(feat['valence'])
    popu = sp.track(i)
    popularity.append(popu['popularity'])

In [96]:
ts = pd.DataFrame({'name':song_name,
                   'album':album,
                   'dance':dance,
                   'acoustic':acoustic,
                   'energy':energy,
                   'instrumental':instrumental,
                   'liveness':liveness,
                   'loudness':loudness,
                   'speech':speech,
                   'tempo':tempo,
                   'valence':valence,
                   'popularity':popularity
    
})

In [97]:
ts.head()

,name,album,dance,acoustic,energy,instrumental,liveness,loudness,speech,tempo,valence,popularity
0,Welcome To New York (Taylor's Version),1989 (Taylor's Version) [Deluxe],0.757,0.009420,0.610,0.000037,0.3670,-4.840,0.0327,116.998,0.685,73
1,Blank Space (Taylor's Version),1989 (Taylor's Version) [Deluxe],0.733,0.088500,0.733,0.000000,0.1680,-5.376,0.0670,96.057,0.701,74
2,Style (Taylor's Version),1989 (Taylor's Version) [Deluxe],0.511,0.000421,0.822,0.019700,0.0899,-4.785,0.0397,94.868,0.305,75
3,Out Of The Woods (Taylor's Version),1989 (Taylor's Version) [Deluxe],0.545,0.000537,0.885,0.000056,0.3850,-5.968,0.0447,92.021,0.206,74
4,All You Had To Do Was Stay (Taylor's Version),1989 (Taylor's Version) [Deluxe],0.588,0.000656,0.721,0.000000,0.1310,-5.579,0.0317,96.997,0.520,72


### Scrape Lyrics from Lyric Genius

In [89]:
#function to scrape lyrics from genius
def scrape_lyrics(artistname, songname):
    artistname2 = str(artistname.replace(' ','-')) if ' ' in artistname else str(artistname)
    songname2 = str(songname.replace(' ','-')) if ' ' in songname else str(songname)
    songname2 = str(songname2.replace('(','')) if '(' in songname2 else str(songname2)
    songname2 = str(songname2.replace(')','')) if ')' in songname2 else str(songname2)
    songname2 = str(songname2.replace("'",'')) if "'" in songname2 else str(songname2)
    page = requests.get('https://genius.com/'+ artistname2 + '-' + songname2 + '-' + 'lyrics')
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics1 = html.find("div", class_="lyrics")
    lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-1 kUgSbL")
    if lyrics1:
        lyrics = lyrics1.get_text()
    elif lyrics2:
        lyrics = lyrics2.get_text()
    elif lyrics1 == lyrics2 == None:
        lyrics = None
    return lyrics

In [90]:
#function to attach lyrics onto data frame
#artist_name should be inserted as a string
def lyrics_onto_frame(df1, artist_name):
    for i,x in enumerate(df1['name']):
        test = scrape_lyrics(artist_name, x)
        df1.loc[i, 'lyrics'] = test
    return df1

In [91]:
ts_lyrics_df = lyrics_onto_frame(ts, 'Taylor Swift')

In [98]:
ts_lyrics_df.head()

,name,album,dance,acoustic,energy,instrumental,liveness,loudness,speech,tempo,valence,popularity,lyrics
0,Welcome To New York (Taylor's Version),1989 (Taylor's Version) [Deluxe],0.757,0.009420,0.610,0.000037,0.3670,-4.840,0.0327,116.998,0.685,73,"[Verse 1]Walkin' through a crowd, the village ..."
1,Blank Space (Taylor's Version),1989 (Taylor's Version) [Deluxe],0.733,0.088500,0.733,0.000000,0.1680,-5.376,0.0670,96.057,0.701,74,"[Verse 1]Nice to meet you, where you been?I co..."
2,Style (Taylor's Version),1989 (Taylor's Version) [Deluxe],0.511,0.000421,0.822,0.019700,0.0899,-4.785,0.0397,94.868,0.305,75,"[Verse 1]MidnightYou come and pick me up, no h..."
3,Out Of The Woods (Taylor's Version),1989 (Taylor's Version) [Deluxe],0.545,0.000537,0.885,0.000056,0.3850,-5.968,0.0447,92.021,0.206,74,[Verse 1]Looking at it nowIt all seems so simp...
4,All You Had To Do Was Stay (Taylor's Version),1989 (Taylor's Version) [Deluxe],0.588,0.000656,0.721,0.000000,0.1310,-5.579,0.0317,96.997,0.520,72,"[Intro](Hey, hey, hey)(Hey, hey, hey)(Hey, hey..."
